In [201]:
# contact: Suchita Kulkarni
#suchita.kulkarni@cern.ch; suchita.kulkarni@gmail.com
#!/usr/bin/env python
import os, sys, shutil
from numpy import *
from pylab import *
from scipy.interpolate import interp1d
import ROOT
#%run plotting_helpers.ipynb
import plotting_helpers



def MakeSinglePlot(infile = '', histnumber = int, groomed_plots = False, scale_au = True, var_legend = True):
    rootfile1 = infile
    histoname = GetAllHistoNames(infile)[0][histnumber]
    histotitle = GetAllHistoNames(infile)[1][histnumber]
    plot_title = "Plots/%s" %(histoname)
    legend_positions = ['r','r','r','r','r','r','l','c','r','r','r','r','r','r','r','r','r','r']

    ROOT.gROOT.SetBatch(True)
    c=ROOT.TCanvas("c","c",800,600)
    f0 = ROOT.TFile(rootfile1,"READ")
    #f0.ls()
    
    # Get the histogram, you can rebin the histogram with the commented out rebin option
    h0=f0.Get(histoname)
    n_bins = h0.GetNbinsX()
    divider = 25 #larger divider gives more bins
    while n_bins%divider != 0 :
        divider += 1
    if divider == n_bins:
        divider = divider/2
    h0.Rebin(int(n_bins/divider))
    #print(n_bins, int(n_bins/divider), h0.GetNbinsX())
    # The following lines will normalise the histogram if scale_au=True and groomed_plots=False
    if scale_au == True and groomed_plots == False:
        norm = h0.GetEntries()
        scale = 1/h0.Integral();
        h0.Scale(scale);
    
    c.cd()
    #Set axis title, you can use latex as demonstrated
    h0.GetXaxis()
    # Thie following lines will set the Y-axis title depending on whether it was normalised
    if scale_au == True and groomed_plots == False:
        h0.GetYaxis().SetTitle('A.U.')
    else:
        h0.GetYaxis()
    #switch off the ugly stats box on the upper right corner
    h0.SetStats(0)
    
    #set label sizes
    h0.GetYaxis().SetLabelSize(0.04)
    h0.GetYaxis().SetTitleSize(0.04)
    h0.GetXaxis().SetTitleSize(0.04)
    h0.GetXaxis().SetLabelSize(0.04)
    h0.SetTitle("")
    #set linewidth and color of the lines
    h0.SetLineWidth(2)
    h0.SetLineColor(ROOT.kBlue-2)
    #actually draw the histogram
    h0.Draw('hist')
    h0.Draw('E1 same')
    
    #set axis offsets so that labels don't get messed up
    h0.GetXaxis().SetTitleOffset(1.4)
    h0.GetYaxis().SetTitleOffset(1.4)
    #set axis ranges
    max_x = h0.GetXaxis().GetXmax()
    min_x = h0.GetXaxis().GetXmin()
    h0.GetXaxis().SetRangeUser(min_x, max_x)
    max_y = h0.GetBinContent(h0.GetMaximumBin())*1.1
    h0.GetYaxis().SetRangeUser(0, max_y)
    #set margins on canvas
    c.SetRightMargin(0.09)
    c.SetLeftMargin(0.15)
    c.SetBottomMargin(0.15)

    # Add a legend and write sample name, information etc. Position dependent on variable legend choice
    latex = ROOT.TLatex ()
    latex.SetNDC(ROOT.kTRUE)
    if var_legend == True:
        legendp = GetLegendPlacement(legend_positions[histnumber])
    else: 
        legendp = GetPosition('r')
    a,b = legendp
    latex.SetTextSize(0.04)
    legend = ROOT.TLegend (a,0.64 ,b,0.74)
    latex.DrawText(a+0.01, 0.85 , "Dark Shower Sample")
    latex.SetTextSize(0.03)
    latex.DrawLatex(a+0.01, 0.8 , "#bf{Higgs events at #sqrt{s}=13 TeV}")
    latex.DrawLatex(a+0.01, 0.76 , "#bf{m_{H}=2 TeV, N_{c}=5, N_{f}=2}")
    legend.AddEntry (histoname, histotitle)
    legend.SetLineWidth (0)
    legend.Draw("same")
    
    # Print statistics
    stat_string = GetStatistics(h0,histotitle)
    print(stat_string)
           
    # Draws groomed histograms in same plot if groomed_plots=True and the groomed histograms exist
    if groomed_plots == True:
        try:
            GetGroomedPlots(f0,c,histoname,h0.GetNbinsX(),legend)
            plot_title = plot_title + "_groomed"
        except ValueError:
            print(sys.exc_info()[0], "occured: No groomed histograms exist of this kind. Will continue without groomed histograms.")
            
    # Save the canvas
    plot_title = plot_title + ".png"
    c.Print(plot_title)
    


#for hist in range(0,18):
#    if hist == 16:
#        continue
#    else:
#        MakeSinglePlot(infile = '/eos/user/n/nhemme/sampleoutput6.root', histnumber = hist)

MakeSinglePlot(infile = '/eos/user/n/nhemme/sampleoutput_noscaling1.root', histnumber = 0, groomed_plots = True, scale_au = True, var_legend = True)


Some Statistics for Lead jet p_{T} with R=1.4: 
    There are 3114 events in the histogram. They are binned into 25 bins. 
    The mean is 888.44 GeV and the standard deviation is 320.94 GeV. 
    Half of the events are contained between 798.62 and 1044.01 GeV.
    
Some Statistics for jet1_pt_softdrop: 
    There are 2469 events in the histogram. They are binned into 25 bins. 
    The mean is 886.12 GeV and the standard deviation is 309.26 GeV. 
    Half of the events are contained between 808.57 and 1042.40 GeV.
    
Some Statistics for jet1_pt_trimmed: 
    There are 1137 events in the histogram. They are binned into 25 bins. 
    The mean is 957.64 GeV and the standard deviation is 372.70 GeV. 
    Half of the events are contained between 853.31 and 1168.85 GeV.
    


Info in <TCanvas::Print>: png file Plots/jet1_pt_groomed.png has been created
